In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:11:44 of 0:11:45 remaining]
Chain 1:  10% [0:00:56 of 0:01:02 remaining]
Chain 1:  20% [0:00:58 of 0:01:12 remaining]
Chain 1:  30% [0:00:51 of 0:01:12 remaining]
Chain 1:  40% [0:00:44 of 0:01:13 remaining]
Chain 1:  50% [0:00:37 of 0:01:13 remaining]
Chain 1:  60% [0:00:29 of 0:01:13 remaining]
Chain 1:  70% [0:00:22 of 0:01:13 remaining]
Chain 1:  80% [0:00:15 of 0:01:13 remaining]
Chain 1:  90% [0:00:07 of 0:01:13 remaining]
Chain 1: 100% [0:00:00 of 0:01:13 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:44 of 0:00:49 remaining]
Chain 2:  20% [0:00:47 of 0:00:58 remaining]
Chain 2:  30% [0:00:43 of 0:01:02 remaining]
Chain 2:  40% [0:00:38 of 0:01:04 remaining]
Chain 2:  50% [0:00:33 of 0:01:06 remaining]
Chain 2:  60% [0:00:27 of 0:01:06 remaining]
Chain 2:  70% [0:00:20 of 0:01:07 remaining]
Chain 2:  80% [0:00:13 of 0:01:07 remaining]
Chain 2:  90% [0:00:07 of 0:01:08 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[25.7439 113.407 0.923656; 23.8119 113.623 0.916686; … ; 25.1869 112.131 0.945099; 30.3224 112.131 0.94574]

[27.7872 116.593 0.859618; 28.5837 116.585 0.851043; … ; 25.8466 114.145 0.906378; 24.1256 114.148 0.89817]

[23.6497 115.235 0.871294; 28.5615 115.23 0.890949; … ; 25.0792 115.853 0.8637; 27.0397 116.459 0.844142]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD       Naive SE       MCSE        ESS   
     s2  26.1405808 1.997937106 0.0121590580 0.0195369117 9000.0000
beta[1] 113.8741291 1.902731671 0.0115796562 0.0340887412 3115.5388
beta[2]   0.9051224 0.041832127 0.0002545822 0.0007394402 3200.4699

Quantiles:
           2.5%       25.0%        50.0%       75.0%        97.5%   
     s2  22.509725  24.7512131  26.03891495  27.4277883  30.31804739
beta[1] 110.168803 112.6046534 113.90578317 115.1437784 117.65732224
beta[2]   0.822729   0.8771617   0.90430177   0.9330343   0.98645237



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*